# Adding Wildlife Images to Raccoon Dataset

Image Data from [here](https://www.kaggle.com/andrewmvd/animal-faces)
- Animal Faces

In [33]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import albumentations as A
import albumentations
from albumentations.pytorch import ToTensorV2

In [2]:
# create a dataframe for our data
data_path = '../DeepLearning_MSDS21/Data/afhq/'

rows = []
for dataset in os.listdir(data_path):
    for label in os.listdir(data_path + f'/{dataset}'):
        for image in os.listdir(data_path + f'/{dataset}' + f'/{label}'):
            row = dict()
            row['image_file'] = image
            row['label'] = label
            row['dataset'] = dataset
        
            # a bit redudant, could build from other data in __getitem__ if wanted
            row['image_path'] = data_path + f'/{dataset}' + f'/{label}'
            rows.append(row)
        
df = pd.DataFrame(rows)
print(len(df))
df.head()

16131


,image_file,label,dataset,image_path
0,pixabay_cat_000455.jpg,cat,train,../DeepLearning_MSDS21/Data/afhq//train/cat
1,pixabay_cat_001993.jpg,cat,train,../DeepLearning_MSDS21/Data/afhq//train/cat
2,pixabay_cat_004633.jpg,cat,train,../DeepLearning_MSDS21/Data/afhq//train/cat
3,pixabay_cat_002242.jpg,cat,train,../DeepLearning_MSDS21/Data/afhq//train/cat
4,pixabay_cat_002524.jpg,cat,train,../DeepLearning_MSDS21/Data/afhq//train/cat


In [5]:
# Select wildlife images, get 148 images to balance raccoon dataset
wild_df = df[(df['label']=='wild') & (df['dataset']=='train')].sample(n=148)
wild_df

,image_file,label,dataset,image_path
11072,flickr_wild_002208.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
10497,pixabay_wild_000453.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
13441,flickr_wild_001095.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
13770,flickr_wild_001280.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
12325,flickr_wild_001592.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
...,...,...,...,...
14600,flickr_wild_000358.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
10029,flickr_wild_001895.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
13352,flickr_wild_000918.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild
11505,flickr_wild_003204.jpg,wild,train,../DeepLearning_MSDS21/Data/afhq//train/wild


In [6]:
# Select columns of interest and rename to match raccoon df
wild_df = wild_df[['image_file', 'label']].rename(columns={'image_file':'filename', 'label':'class'})
wild_df

,filename,class
11072,flickr_wild_002208.jpg,wild
10497,pixabay_wild_000453.jpg,wild
13441,flickr_wild_001095.jpg,wild
13770,flickr_wild_001280.jpg,wild
12325,flickr_wild_001592.jpg,wild
...,...,...
14600,flickr_wild_000358.jpg,wild
10029,flickr_wild_001895.jpg,wild
13352,flickr_wild_000918.jpg,wild
11505,flickr_wild_003204.jpg,wild


In [15]:
# Train/valid split Wildlife
valid_w = wild_df.sample(frac=.2)
train_w = wild_df.drop(valid_w.index)
len(train_w),len(valid_w)

(118, 30)

In [23]:
# Raccoon df
raccoon_df = pd.read_csv('./Data/train_labels_.csv')
# Remove images to >1 raccoon
new = pd.DataFrame(raccoon_df.filename.value_counts() > 1).reset_index()
new = new[new['filename'] == True]
for_removal = list(new['index'].values)
raccoon_df = raccoon_df[~raccoon_df['filename'].isin(for_removal)]

In [24]:
# Rescale bounding box for raccoon images
image_size = 128
raccoon_df['xmin_rs'] = raccoon_df['xmin']*image_size/raccoon_df['width']
raccoon_df['xmax_rs'] = raccoon_df['xmax']*image_size/raccoon_df['width']
raccoon_df['ymin_rs'] = raccoon_df['ymin']*image_size/raccoon_df['height']
raccoon_df['ymax_rs'] = raccoon_df['ymax']*image_size/raccoon_df['height']
raccoon_df

,filename,width,height,class,xmin,ymin,xmax,ymax,xmin_rs,xmax_rs,ymin_rs,ymax_rs
0,raccoon-17.jpg,259,194,raccoon,95,60,167,118,46.949807,82.532819,39.587629,77.855670
1,raccoon-11.jpg,660,432,raccoon,3,1,461,431,0.581818,89.406061,0.296296,127.703704
4,raccoon-60.jpg,273,185,raccoon,58,33,197,127,27.194139,92.366300,22.832432,87.870270
5,raccoon-69.jpg,205,246,raccoon,12,11,188,240,7.492683,117.385366,5.723577,124.878049
6,raccoon-180.jpg,600,400,raccoon,119,21,368,399,25.386667,78.506667,6.720000,127.680000
...,...,...,...,...,...,...,...,...,...,...,...,...
168,raccoon-137.jpg,320,240,raccoon,71,8,304,233,28.400000,121.600000,4.266667,124.266667
169,raccoon-171.jpg,224,225,raccoon,108,21,180,115,61.714286,102.857143,11.946667,65.422222
170,raccoon-190.jpg,259,194,raccoon,78,54,153,135,38.548263,75.613900,35.628866,89.072165
171,raccoon-22.jpg,640,360,raccoon,252,76,466,335,50.400000,93.200000,27.022222,119.111111


In [25]:
# Train/valid split Raccoon
valid_r = raccoon_df.sample(frac=.2)
train_r = raccoon_df.drop(valid_r.index)
len(train_r),len(valid_r)

(118, 30)

In [26]:
# Combine dfs
train = train_r.append(train_w, ignore_index=True)
valid = valid_r.append(valid_w, ignore_index=True)
len(train), len(valid)

(236, 60)

In [29]:
# Fill NAs for bb with 0
train[['xmin_rs','ymin_rs','xmax_rs','ymax_rs']] = train[['xmin_rs','ymin_rs','xmax_rs','ymax_rs']].fillna(value=0)
valid[['xmin_rs','ymin_rs','xmax_rs','ymax_rs']] = valid[['xmin_rs','ymin_rs','xmax_rs','ymax_rs']].fillna(value=0)
train

,filename,width,height,class,xmin,ymin,xmax,ymax,xmin_rs,xmax_rs,ymin_rs,ymax_rs
0,raccoon-17.jpg,259.0,194.0,raccoon,95.0,60.0,167.0,118.0,46.949807,82.532819,39.587629,77.855670
1,raccoon-11.jpg,660.0,432.0,raccoon,3.0,1.0,461.0,431.0,0.581818,89.406061,0.296296,127.703704
2,raccoon-60.jpg,273.0,185.0,raccoon,58.0,33.0,197.0,127.0,27.194139,92.366300,22.832432,87.870270
3,raccoon-69.jpg,205.0,246.0,raccoon,12.0,11.0,188.0,240.0,7.492683,117.385366,5.723577,124.878049
4,raccoon-180.jpg,600.0,400.0,raccoon,119.0,21.0,368.0,399.0,25.386667,78.506667,6.720000,127.680000
...,...,...,...,...,...,...,...,...,...,...,...,...
231,flickr_wild_001243.jpg,NaN,NaN,wild,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
232,flickr_wild_000358.jpg,NaN,NaN,wild,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
233,flickr_wild_000918.jpg,NaN,NaN,wild,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
234,flickr_wild_003204.jpg,NaN,NaN,wild,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000


In [25]:
# !mkdir ../DeepLearning_MSDS21/Data/afhq/train/wild_rs

In [11]:
# Resize Wildlife images before training
def resize_img(path, newpath, size):
    img = cv2.imread(path)
    
    start = time.time()
    img = cv2.resize(img, size)
    end = time.time()
    
    cv2.imwrite(newpath, img)
    return end - start

# resize all of the images
total_time_resize = 0.0
image_size = 128
for idx in range(len(wild_df)):
    row = wild_df.iloc[idx]
    fname = row['filename']
    data_path = '../DeepLearning_MSDS21/Data/afhq/train/wild'
    path = data_path + '/' + row['filename']
    newpath = data_path + '_rs/'+fname
    
    total_time_resize += resize_img(path, newpath, (image_size, image_size))
total_time_resize

0.013692378997802734

In [12]:
# Check that resize worked
path = data_path + '_rs/' + wild_df.iloc[0]['filename']
img = cv2.imread(path)
img.shape

(128, 128, 3)

In [48]:
class AnimalDataset(Dataset):
    def __init__(self, df, augment=True):
        # Dataframe with bounding boxes
        self.df = df
        
        # define the transformation
        if augment == True:
            self.transforms = A.Compose([
                albumentations.augmentations.transforms.VerticalFlip(p=.25),
                albumentations.augmentations.transforms.HorizontalFlip(p=.25),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2()])
        else:
            self.transforms = A.Compose([
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                
                # convert to a tensor and move color channels
                ToTensorV2()
            ])
        
    def __len__(self):
        return len(self.df) # this only works if 1:1 image:label
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        class_dict = {'wild':0, 'raccoon':1}
        class_label = class_dict[row['class']]
        
        # Create image array
        fname = row['filename']
        if class_label == 1:
            raccoon_path = './Data/Racoon Images/images_rs'
            img_path = raccoon_path + '/' + fname
        else:
            wild_path = '../DeepLearning_MSDS21/Data/afhq/train/wild_rs'
            img_path = wild_path + '/' + fname
        img = cv2.imread(img_path)
        
        # convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        transformed = self.transforms(image=img.astype(np.uint8))
        img = transformed['image']   
        
        
        # Get bounding box label
        xmin, ymin = tuple(list(row[['xmin_rs','ymin_rs']].values))
        xmax, ymax = tuple(list(row[['xmax_rs','ymax_rs']].values))
        bbox_label = torch.tensor([xmin,ymin,xmax,ymax]).float()
        
        return img, class_label, bbox_label

In [49]:
train_ds = AnimalDataset(train)
valid_ds = AnimalDataset(valid)

In [50]:
next(iter(train_ds))

(tensor([[[-0.5596, -1.9809, -1.9467,  ...,  0.3309,  0.1254,  0.7077],
          [-0.6452, -2.1179, -1.8097,  ..., -0.4568, -0.0458,  0.2624],
          [-1.3473, -1.7925, -1.7240,  ..., -0.5253, -0.2513, -0.2513],
          ...,
          [-0.0629, -0.4397, -0.1657,  ..., -2.1179, -0.9877, -0.8335],
          [ 0.2111,  0.0398, -0.1314,  ..., -1.7412, -1.8953, -0.6109],
          [-0.0801,  0.0741, -0.0801,  ..., -1.4500, -1.0390,  0.2967]],
 
         [[-0.2500, -1.7031, -1.7031,  ...,  1.3431,  1.0980,  1.6758],
          [-0.3375, -1.8957, -1.5630,  ...,  0.5553,  0.9405,  1.2556],
          [-1.0903, -1.5455, -1.5280,  ...,  0.5203,  0.7829,  0.7654],
          ...,
          [ 0.8354,  0.4503,  0.7479,  ..., -1.3704, -0.0924,  0.1176],
          [ 1.1155,  0.9580,  0.7829,  ..., -0.9328, -0.9678,  0.3978],
          [ 0.8179,  0.9930,  0.8354,  ..., -0.6352, -0.0924,  1.3606]],
 
         [[-0.1661, -1.6127, -1.5256,  ...,  0.7054,  0.6182,  1.2631],
          [-0.2532, -1.7522,